In [ ]:
!pip install qiskit
!pip install pylatexenc
!pip install qiskit.ignis
!pip install qiskit_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 18.8 MB/s eta 0:00:00


In [ ]:
from qiskit import *
# Qiskit module
from qiskit import QuantumCircuit
import qiskit.circuit.library as circuit_library
import qiskit.quantum_info as qi
#from qiskit import execute
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import EfficientSU2
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from qiskit.circuit import ParameterVector
from torch import Tensor
import torch.nn as nn
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/MyDrive/New_test/'

# Load Dataset

In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing import sequence

vocab_size = 1000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=vocab_size)

class_names = ["Negative", "Positive"]

17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# Select dataset evenly 
selected_num=128

new_x_train=[]
new_y_train=[]
indx=0
while len(new_x_train) < selected_num:
    if len(x_train[indx]) <= 200:
        sum_y=np.sum(new_y_train)
        len_y=len(new_y_train)
        if sum_y >= selected_num//2:
            if y_train[indx]==0:
                new_x_train.append(x_train[indx])
                new_y_train.append(y_train[indx])
                indx+=1
            else:
                indx+=1
                continue
        elif ((len_y-sum_y) >= selected_num//2):
            if y_train[indx]==1:
                new_x_train.append(x_train[indx])
                new_y_train.append(y_train[indx])
                indx+=1
            else:
                indx+=1
                continue
        else:
            new_x_train.append(x_train[indx])
            new_y_train.append(y_train[indx])
            indx+=1
    else:
        indx+=1

selected_num=128

new_x_test=[]
new_y_test=[]
indx=0
while len(new_x_test) < selected_num:
    if len(x_test[indx]) <= 200:
        sum_y=np.sum(new_y_test)
        len_y=len(new_y_test)
        if sum_y >= selected_num//2:
            if y_test[indx]==0:
                new_x_test.append(x_test[indx])
                new_y_test.append(y_test[indx])
                indx+=1
            else:
                indx+=1
                continue
        elif ((len_y-sum_y) >= selected_num//2):
            if y_test[indx]==1:
                new_x_test.append(x_test[indx])
                new_y_test.append(y_test[indx])
                indx+=1
            else:
                indx+=1
                continue
        else:
            new_x_test.append(x_test[indx])
            new_y_test.append(y_test[indx])
            indx+=1
    else:
        indx+=1

In [ ]:
x_train=np.array(new_x_train)#[:128]
y_train=np.array(new_y_train)#[:128]

x_test=np.array(new_x_test)#[:128]
y_test=np.array(new_y_test)#[:128]

<ipython-input-7-b98d59874a29>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train=np.array(new_x_train)#[:128]
<ipython-input-7-b98d59874a29>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test=np.array(new_x_test)#[:128]


In [ ]:
# Get the word index from the dataset
word_index = tf.keras.datasets.imdb.get_word_index()

# Ensure that "special" words are mapped into human readable terms 
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNKNOWN>"] = 2
word_index["<UNUSED>"] = 3

# Perform reverse word lookup and make it callable
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
# Concatonate test and training datasets
allreviews = np.concatenate((x_train, x_test), axis=0)

# Review lengths across test and training whole datasets
print("Maximum review length: {}".format(len(max((allreviews), key=len))))
print("Minimum review length: {}".format(len(min((allreviews), key=len))))
result = [len(x) for x in allreviews]
print("Mean review length: {}".format(np.mean(result)))

# Print a review and it's class as stored in the dataset. Replace the number
# to select a different review.
print("")
print("Machine readable Review")
print("  Review Text: " + str(x_train[60]))
print("  Review Sentiment: " + str(y_train[60]))

# Print a review and it's class in human readable format. Replace the number
# to select a different review.
print("")
print("Human Readable Review")
print("  Review Text: " + decode_review(x_train[0]))
print("  Review Sentiment: " + class_names[y_train[0]])

Maximum review length: 200
Minimum review length: 19
Mean review length: 132.546875

Machine readable Review
  Review Text: [1, 13, 244, 6, 666, 337, 7, 2, 2, 5, 28, 296, 958, 7, 27, 102, 366, 150, 5, 600, 7, 98, 28, 2, 8, 97, 72, 462, 2, 122, 89, 8, 2, 369, 5, 2, 84, 10, 10, 14, 20, 9, 2, 44, 6, 132, 52, 17, 2, 84, 125, 190, 29, 47, 35, 2, 270, 7, 2, 15, 2, 90, 39, 399, 183, 15, 238, 43, 97, 90, 804, 21, 11, 4, 130, 29, 630, 56, 399, 98, 4, 2, 7, 113, 10, 10, 4, 20, 9, 73, 2, 19, 486, 883, 52, 116, 5, 82, 6, 227, 7, 6, 2, 837, 94, 31, 7, 148, 102, 121, 25, 100, 43, 30, 654, 54, 12, 630]
  Review Sentiment: 1

Human Readable Review
  Review Text: <START> big <UNKNOWN> big <UNKNOWN> bad music and a <UNKNOWN> <UNKNOWN> <UNKNOWN> these are the words to best <UNKNOWN> this terrible movie i love cheesy horror movies and i've seen <UNKNOWN> but this had got to be on of the worst ever made the plot is <UNKNOWN> <UNKNOWN> and ridiculous the acting is an <UNKNOWN> the script is completely <UNKN

In [ ]:
# The length of reviews
review_length = 200

# Padding / truncated our reviews
x_train = sequence.pad_sequences(x_train, maxlen = review_length)
x_test = sequence.pad_sequences(x_test, maxlen = review_length)

# Check the size of our datasets. Review data for both test and training should 
# contain 25000 reviews of 500 integers. Class data should contain 25000 values, 
# one for each review. Class values are 0 or 1, indicating a negative 
# or positive review.
print("Shape Training Review Data: " + str(x_train.shape))
print("Shape Training Class Data: " + str(np.shape(y_train)))
print("Shape Test Review Data: " + str(x_test.shape))
print("Shape Test Class Data: " + str(y_test.shape))

# Note padding is added to start of review, not the end
print("")
print("Human Readable Review Text (post padding): " + decode_review(x_train[0]))

Shape Training Review Data: (128, 200)
Shape Training Class Data: (128,)
Shape Test Review Data: (128, 200)
Shape Test Class Data: (128,)

Human Readable Review Text (post padding): <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> big <UNKNOWN> big <UNKNOWN> bad music and a <UNKNOWN> <UNKNOWN> <UNKNOWN> these are the words to best <UNKNOWN> this terrible movie i love cheesy horror movies and i've seen <UNKNOWN> but this had got to be on of the worst ever made the plot is <UNKNOWN> <UNKNOWN> and ridiculous the acting is an <UNKNOWN> the script is completely <UNKNOWN> the best is the end <UNKNOWN> with the <UNKNOWN> and how he worked out who the killer is it's just so <UNKNOWN> <UNKNOWN> written the <UNKNOWN> are <UNKNOWN> and funny in <UNKNOWN> <UNKNOWN> the <UNKNOWN> is big lots of <UNKNOWN> <UNKNOWN> men <UNKNOWN> those cut <UNKNOWN> <UNKNOWN> that show off their <UNKNOWN> <UNKNOWN> that men actually <UNKNOWN> them and the music is just <UNKNOWN> <UNKNOWN> tha

In [ ]:
new_X_train=[]
for seq in x_train:
    a=torch.tensor(np.pi*(seq/1000-0.5))
    new_X_train.append(torch.reshape(a, (10, 20)))

new_y_train=[]
for seq in y_train:
    new_y_train.append(torch.tensor(seq))

In [ ]:
new_X_train=torch.stack(new_X_train)

label=torch.stack(new_y_train)

In [ ]:
new_X_test=[]
for seq in x_test:
    a=torch.tensor(np.pi*(seq/1000-0.5))
    new_X_test.append(torch.reshape(a, (10, 20)))

new_y_test=[]
for seq in y_test:
    new_y_test.append(torch.tensor(seq))

# Model

In [ ]:
import random
class QSAL_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.seq_num=S
        self.init_params_Q=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_K=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.init_params_V=torch.nn.Parameter(torch.stack([(np.pi/4) * (2 * torch.randn(n*(D+2)) - 1) for _ in range(self.seq_num)]))
        self.params_input=[ParameterVector('IN'+str(i),n*(Denc+2)) for i in range(self.seq_num)]
        self.params_Q=[ParameterVector('Q'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_K=[ParameterVector('K'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.params_V=[ParameterVector('V'+str(i),n*(D+2)) for i in range(self.seq_num)]
        self.num_q=n
        self.Denc=Denc
        self.D=D
        self.d=n*(Denc+2)
        self.bit_string_Z=SparsePauliOp.from_list([('I'*(self.num_q-1)+'Z', 1)])
        self.pauli_strings=[]
        for i in range(self.d):
            string=['I']*self.num_q
            while string==['I']*self.num_q:
                for j in range(self.num_q):
                    a=random.randint(0, 4)
                    if a==0:
                        continue
                    elif a==1:
                        string[j]='X'
                    elif a==2:
                        string[j]='Y'
                    elif a==3:
                        string[j]='Z'
            self.pauli_strings.append(SparsePauliOp.from_list([("".join(string), 1)]))  

        Q_qnn=[EstimatorQNN(circuit=self.QSAL_cir("Q",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_Q[i]) for i in range(self.seq_num)]
        K_qnn=[EstimatorQNN(circuit=self.QSAL_cir("K",i),observables=[self.bit_string_Z], input_params=self.params_input[i], weight_params=self.params_K[i]) for i in range(self.seq_num)]
        V_qnn=[EstimatorQNN(circuit=self.QSAL_cir("V",i),observables=self.pauli_strings, input_params=self.params_input[i], weight_params=self.params_V[i]) for i in range(self.seq_num)]

        self.Q_models=[TorchConnector(Q_qnn[i], initial_weights=self.init_params_Q[i]) for i in range(self.seq_num)]
        self.K_models=[TorchConnector(K_qnn[i], initial_weights=self.init_params_K[i]) for i in range(self.seq_num)]
        self.V_models=[TorchConnector(V_qnn[i], initial_weights=self.init_params_V[i]) for i in range(self.seq_num)]

    def QSAL_cir(self,type,indx):

        qc=QuantumCircuit(self.num_q)  
        if type=="Q":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_Q[indx])
            
        elif type=="K":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_K[indx])

        elif type=="V":
            self.Feature_map(qc,self.params_input[indx])
            self.ansatz(qc,self.params_V[indx])

        return qc

    def Feature_map(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.Denc):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)

            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1
            

    def ansatz(self,qc,params):
        indx=0
        for j in range(self.num_q):
            qc.rx(params[indx],j)
            qc.ry(params[indx+1],j)
            indx+=2
        for i in range(self.D):
            for j in range(self.num_q):
                qc.cx(j,(j+1)%self.num_q)
                
            for j in range(self.num_q):
                #qc.rx(params[indx],j)
                qc.ry(params[indx],j)
                indx+=1

    def forward(self,input):

        Q_output=torch.stack([self.Q_models[i](input[:,i]) for i in range(self.seq_num)])
        K_output=torch.stack([self.K_models[i](input[:,i]) for i in range(self.seq_num)])
        V_output=torch.stack([self.V_models[i](input[:,i]) for i in range(self.seq_num)])
        batch_size=len(input)
        Q_output=Q_output.transpose(0,2).repeat((self.seq_num,1,1))
        K_output=K_output.transpose(0,2).repeat((self.seq_num,1,1)).transpose(0,2)
        #print(V_output.size())
        #Q_grid, K_grid=torch.meshgrid(Q_output, K_output, indexing='ij')
        alpha=torch.exp(-(Q_output-K_output)**2)
        alpha=alpha.transpose(0,1)
        V_output=V_output.transpose(0,1)
        output=[]

        for i in range(self.seq_num):
            
            Sum_a=torch.sum(alpha[:,i,:],-1)
            div_sum_a=(1/Sum_a).repeat(self.d,self.seq_num,1).transpose(0,2)
            
            Sum_w=torch.sum(alpha[:,:,i].repeat((self.d,1,1)).transpose(0,2).transpose(0,1)*V_output*div_sum_a,1)
            output.append(Sum_w)
        return input+torch.stack(output).transpose(0,1)


class QSANN_qiskit(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.qsal_lst=[QSAL_qiskit(S,n,Denc,D) for _ in range(num_layers)]
        self.qnn=nn.Sequential(*self.qsal_lst)

    def forward(self,input):

        return self.qnn(input)
        
class QSANN_text_classifier(torch.nn.Module):
    def __init__(self,S,n,Denc,D,num_layers):
        """
        # input: input data
        # weight: trainable parameter
        # n: # of of qubits
        # d: embedding dimension which is equal to n(Denc+2)
        # Denc: the # number of layers for encoding 
        # D: the # of layers of variational layers
        # type "K": key, "Q": Query, "V": value
        """
        super().__init__()
        self.Qnn=QSANN_qiskit(S,n,Denc,D,num_layers)
        self.final_layer=nn.Linear(n*(Denc+2)*S, 1)
        self.final_layer=self.final_layer.float()

    def forward(self,input):

        x=self.Qnn(input)
        x=torch.flatten(x,start_dim=1)
        
        return torch.sigmoid(self.final_layer(x))

# Training (Random Toy data)

In [ ]:
X=[[np.array([random.random() for _ in range(6)]) for _ in range(5)] for _ in range(32)]
Y=[random.randint(0,1) for _ in range(32)]


In [ ]:
model=QSANN_text_classifier(5,2,1,1,1)

In [ ]:
optimizer = torch.optim.Adam(lr=0.03, params=model.parameters())

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

481


In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
for iepoch in tqdm(range(10)):
    optimizer.zero_grad()
    X_tensor=torch.tensor(X)
    predictions=model(X_tensor.float()).squeeze(1)
    #predictions=torch.sign(predictions)
    label=torch.tensor(Y)
    loss = criterion(predictions, label.float())
    acc = binary_accuracy(predictions, label)

    loss.backward()
    optimizer.step()
    #print(acc)
    #print(loss)

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-16-5cfb7c505ff2>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X_tensor=torch.tensor(X)
100%|██████████| 10/10 [05:23<00:00, 32.31s/it]


# Train Real Data

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = (torch.round(torch.sign(preds-0.5))+1)//2
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
model = QSANN_text_classifier(10,2,8,1,1)

/usr/local/lib/python3.10/dist-packages/qiskit_machine_learning/connectors/torch_connector.py:312: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights.data = torch.tensor(initial_weights, dtype=torch.float)


In [ ]:
optimizer = torch.optim.Adam(lr=0.005, params=model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_params)

381


In [ ]:

for iepoch in tqdm(range(40)):
    
    optimizer.zero_grad()

    idx = torch.randperm(128)[:32]
    X_batch = new_X_train[idx]
    predictions=model(X_batch.float()).squeeze(1)

    label_batch=label[idx]
    loss = criterion(predictions, label_batch.float())
    acc = binary_accuracy(predictions,label_batch)
    print('')
    print('Accuracy:',acc)
    print('')
    print(loss)
    loss.backward()
    optimizer.step()

  0%|          | 0/40 [00:00<?, ?it/s]


Accuracy: tensor(0.5938)

tensor(44.9861, grad_fn=<DivBackward1>)


  2%|▎         | 1/40 [08:25<5:28:46, 505.82s/it]


Accuracy: tensor(0.5312)

tensor(52.1053, grad_fn=<DivBackward1>)


  5%|▌         | 2/40 [16:51<5:20:16, 505.70s/it]

...

 95%|█████████▌| 38/40 [5:22:49<16:58, 509.05s/it]


Accuracy: tensor(0.8125)

tensor(53.7304, grad_fn=<DivBackward1>)


 98%|█████████▊| 39/40 [5:31:19<08:29, 509.25s/it]


Accuracy: tensor(0.7812)

tensor(53.9442, grad_fn=<DivBackward1>)


100%|██████████| 40/40 [5:39:59<00:00, 509.98s/it]


In [ ]:
X_batch = new_X_train
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label.float())
acc = binary_accuracy(predictions,label)
print('')
print('Accuracy:',acc)
print('')
print(loss)


Accuracy: tensor(0.7969)

tensor(302.9272, grad_fn=<DivBackward1>)


In [ ]:
label_test=torch.stack(new_y_test)

In [ ]:
X_batch = torch.stack(new_X_test)
predictions=model(X_batch.float()).squeeze(1)

loss = criterion(predictions, label_test.float())
acc = binary_accuracy(predictions,label_test)
print('')
print('Accuracy:',acc)
print('')
print(loss)


Accuracy: tensor(0.4219)

tensor(312.8112, grad_fn=<DivBackward1>)


# Save Model

In [ ]:
file_name=path+'QSANN_qiskit_model_1'
torch.save(model.state_dict(),file_name)